In [238]:
import numpy as np
import scipy as sp
import math
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1, kernel="linear"),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)]

Classifiers = ["KNN", "SVM", "GP", "DT", "RF", "XGB"]
beta = "XGB"

parameters = {"KNN":{"n_neighbors":[], "n_jobs":-1},
              "SVM":{"C":[],"kernel":[],"degree":[],"probability":True},
              "GP":{"kernel":[],"n_restarts_optimizer":[], "n_jobs":-1},
              "DT":{"splitter":[], "max_depth":[]},
              "RF":{"n_estimators":[], "max_depth":[] },
              "XGB":{"learning_rate":[], "n_estimators":[], "max_depth":[],"min_child_weight":[], "nthread":8}
            }

In [239]:
def sample_hp_Ai(Ai):
    if(Ai is "KNN"):
        parameters["KNN"]["n_neighbors"].append(math.ceil(np.random.uniform(100)))
        return(parameters["KNN"])
    elif(Ai is "SVM"):
        parameters["SVM"]["C"].append(float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0)))))
        parameters["SVM"]["kernel"].append(np.random.choice(["linear", "rbf"]))
        parameters["SVM"]["degree"].append(np.random.randint(1,4))
        return(parameters["SVM"])
    elif(Ai is "GP"):
        parameters["GP"]["kernel"].append(np.random.choice(["linear", "rbf"]))
        parameters["GP"]["n_restarts_optimizer"].append(np.random.randint(1,10))
        return(parameters["GP"])
    elif(Ai is "DT"):
        parameters["DT"]["splitter"].append(np.random.choice(["best", "random"]))
        parameters["DT"]["max_depth"].append(np.random.randint(1,100))
        return(parameters["DT"])
    elif(Ai is "RF"):
        parameters["RF"]["n_estimators"].append(np.random.randint(1,20))
        parameters["RF"]["max_depth"].append(np.random.randint(1,100))
        return(parameters["RF"])
    elif(Ai is "XGB"):
        parameters["XGB"]["learning_rate"].append(float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0)))))
        parameters["XGB"]["n_estimators"].append(np.random.randint(800,2000))
        parameters["XGB"]["max_depth"].append(np.random.randint(5,200))
        parameters["XGB"]["min_child_weight"].append(np.random.randint(1,10))
        return(parameters["XGB"])

In [240]:
def get_params(A, B, P, N):
    mu = np.random.dirichlet(P)
    N = [int(mu_i*N) for mu_i in mu]
    [sample_hp_Ai(Classifiers[i]) for i in range(len(Classifiers)) for j in range(N[i])]
    Phi = sample_hp_Ai("XGB")
    Psi = [parameters[key] for key in parameters if key is not "XGB"]
    return Phi, N, Psi

In [241]:
get_params(Classifiers, "RF", P=(np.arange(1,len(Classifiers)+1)*100)[::-1], N=10)

({'learning_rate': [1.54],
  'max_depth': [70],
  'min_child_weight': [9],
  'n_estimators': [1474],
  'nthread': 8},
 [2, 2, 1, 1, 0, 0],
 [{'n_jobs': -1, 'n_neighbors': [39, 53]},
  {'C': [1.15, 0.92],
   'degree': [2, 1],
   'kernel': ['rbf', 'rbf'],
   'probability': True},
  {'kernel': ['linear'], 'n_jobs': -1, 'n_restarts_optimizer': [5]},
  {'max_depth': [46], 'splitter': ['random']},
  {'max_depth': [], 'n_estimators': []}])

In [242]:
for key in parameters:
    print(key,parameters[key])

KNN {'n_neighbors': [39, 53], 'n_jobs': -1}
SVM {'C': [1.15, 0.92], 'kernel': ['rbf', 'rbf'], 'degree': [2, 1], 'probability': True}
GP {'kernel': ['linear'], 'n_restarts_optimizer': [5], 'n_jobs': -1}
DT {'splitter': ['random'], 'max_depth': [46]}
RF {'n_estimators': [], 'max_depth': []}
XGB {'learning_rate': [1.54], 'n_estimators': [1474], 'max_depth': [70], 'min_child_weight': [9], 'nthread': 8}
